In [25]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [44]:
out='../output/'
inb='../input/flows/'
pref='idm'
inp = inb + pref + '/'

In [45]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2021-04-01,94,MOZN_EDM,0,ZIMA
1,2021-04-01,94,ZIMA,0,BOT
2,2021-04-01,47,MOZN_EDM,1,ZIMA
3,2021-04-01,47,ZIMA,1,BOT
4,2021-04-01,56,MOZN_EDM,2,ZIMA
...,...,...,...,...,...
445,2022-07-31,155,RSAN,21,BOT
446,2022-07-31,125,MOZS,22,RSAN
447,2022-07-31,125,RSAN,22,BOT
448,2022-07-31,125,MOZS,23,RSAN


In [46]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

C:\Users\kabom\AppData\Local\Temp\ipykernel_21532\1849715270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dam=dam.drop(['date'],1)


price  volume  year  month  day  day_of_week    value  cat
date       hour                                                            
2017-05-04 4       0.4     0.5  2017      5    4            3      0.2  off
           5       0.0     0.0  2017      5    4            3      0.0  off
           6       0.0     0.0  2017      5    4            3      0.0  off
           7       0.0     0.0  2017      5    4            3      0.0  off
           8       1.5     8.0  2017      5    4            3     12.0  off
...                ...     ...   ...    ...  ...          ...      ...  ...
2022-07-15 19    300.0    43.0  2022      7   15            4  12900.0  off
           20     85.0     0.0  2022      7   15            4      0.0  off
           21      0.0     0.0  2022      7   15            4      0.0  off
           22      0.0     0.0  2022      7   15            4      0.0  off
           23      0.0     0.0  2022      7   15            4      0.0  off

[43764 rows x 8 columns]

In [47]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

C:\Users\kabom\AppData\Local\Temp\ipykernel_21532\2288514685.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  flow=flow.dropna(0)


,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2021-04-01,4,65,MOZN_EDM,ZIMA,2021,4,1,3,30.0,1950.0
1,2021-04-01,4,65,ZIMA,BOT,2021,4,1,3,30.0,1950.0
2,2021-04-01,5,54,MOZN_EDM,ZIMA,2021,4,1,3,30.0,1620.0
3,2021-04-01,5,54,ZIMA,BOT,2021,4,1,3,30.0,1620.0
4,2021-04-01,7,74,MOZS,RSAN,2021,4,1,3,75.0,5550.0
...,...,...,...,...,...,...,...,...,...,...,...
7209,2022-07-15,18,29,RSAN,BOT,2022,7,15,4,300.0,8700.0
7210,2022-07-15,19,43,MOZS,RSAN,2022,7,15,4,300.0,12900.0
7211,2022-07-15,19,43,RSAN,BOT,2022,7,15,4,300.0,12900.0
7212,2022-07-15,20,41,MOZS,RSAN,2022,7,15,4,85.0,3485.0


In [48]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2021-04-01,4,65,1950.0,30.0,ZIMA,2021,4,1,3
1,2021-04-01,4,65,1950.0,30.0,BOT,2021,4,1,3
2,2021-04-01,5,54,1620.0,30.0,ZIMA,2021,4,1,3
3,2021-04-01,5,54,1620.0,30.0,BOT,2021,4,1,3
4,2021-04-01,7,74,5550.0,75.0,RSAN,2021,4,1,3
...,...,...,...,...,...,...,...,...,...,...
7209,2022-07-15,18,-29,-8700.0,300.0,RSAN,2022,7,15,4
7210,2022-07-15,19,-43,-12900.0,300.0,MOZS,2022,7,15,4
7211,2022-07-15,19,-43,-12900.0,300.0,RSAN,2022,7,15,4
7212,2022-07-15,20,-41,-3485.0,85.0,MOZS,2022,7,15,4


In [49]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [50]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week  index  flow    value  \
date       hour                                                             
2021-04-01 4     BOT  2021      4    1            3      1    65   1950.0   
           4     MOZ  2021      4    1            3      0   -65  -1950.0   
           5     BOT  2021      4    1            3      3    54   1620.0   
           5     MOZ  2021      4    1            3      2   -54  -1620.0   
           7     BOT  2021      4    1            3      5    74   5550.0   
...              ...   ...    ...  ...          ...    ...   ...      ...   
2022-07-15 18    MOZ  2022      7   15            4   7208   -29  -8700.0   
           19    BOT  2022      7   15            4   7211    43  12900.0   
           19    MOZ  2022      7   15            4   7210   -43 -12900.0   
           20    BOT  2022      7   15            4   7213    41   3485.0   
           20    MOZ  2022      7   15            4   7212   -41  -3485.0   

                 price  cat  
date       hour              
2021-04-01 4      30.0  off  
           4      30.0  off  
           5      30.0  off  
           5      30.0  off  
           7      75.0  off  
...                ...  ...  
2022-07-15 18    300.0  off  
           19    300.0  off  
           19    300.0  off  
           20     85.0  off  
           20     85.0  off  

[7526 rows x 10 columns]

In [51]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat side                                       
2021 4     BOT  off consume      8099  508776.53         158   
           ESW  off consume       186    6231.60           6   
           LES  off supply         -5    -300.00           1   
           MOZ  off supply      -4950 -333376.30         120   
           NAM  off supply       -591  -63619.70          20   
...                               ...        ...         ...   
2022 7     BOT  off supply       -155   -3875.00           7   
           ESW  off consume      1248   93685.50          42   
           LES  off consume       143   37821.08           3   
           MOZ  off supply      -1496 -188509.30          47   
           ZAMZ off supply       -499 -135621.08          12   

                             price_weight_mean  price_mean      power  
year month node cat side                                               
2021 4     BOT  off consume          62.819673   67.016835  51.259494  
           ESW  off consume          33.503226   33.493333  31.000000  
           LES  off supply           60.000000   60.000000  -5.000000  
           MOZ  off supply           67.348747   71.458417 -41.250000  
           NAM  off supply          107.647547  102.023500 -29.550000  
...                                        ...         ...        ...  
2022 7     BOT  off supply           25.000000   25.000000 -22.142857  
           ESW  off consume          75.068510   61.954762  29.714286  
           LES  off consume         264.483077  264.580000  47.666667  
           MOZ  off supply          126.008890  113.422128 -31.829787  
           ZAMZ off supply          271.785731  276.145000 -41.583333  

[113 rows x 6 columns]

In [52]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')